In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive/') # this makes your Drive appear in 'content' folder
df=pd.read_csv('/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/test.csv',
                         index_col=0)
print(df.size)

Mounted at /content/drive/


<ipython-input-2-8b80872d1773>:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/test.csv',


24655512


dense columns: rating_y,loves_count,reviews

categorical columns: everything else

label:rating_x

In [3]:
def create_user_info(skin_tone_quartile_num, skin_type, hair_color, skin_tone_num,
       eye_color_num, skin_type_num, hair_color_num,
       skin_tone_quartile_num1):
  user_info=pd.DataFrame({"skin_tone_quartile_num": [skin_tone_quartile_num],
                          "skin_type":[skin_type],
                          "hair_color":[hair_color],
                          "skin_tone_num":[skin_tone_num],
                          "eye_color_num":[eye_color_num],
                          "skin_type_num":[skin_type_num],
                          "hair_color_num":[hair_color_num],
                          "skin_tone_quartile_num.1":[skin_tone_quartile_num1]})
  return user_info
user_info=create_user_info("1.0","oily","black","3.0","5.0","1.0","7.0","1.0")
product_df=pd.read_csv("/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/Data/product_info_clean.csv")
user_info=pd.concat([user_info] * len(product_df), ignore_index=True)
result_df = pd.concat([user_info, product_df], axis=1)

In [4]:
result_df.columns

Index(['skin_tone_quartile_num', 'skin_type', 'hair_color', 'skin_tone_num',
       'eye_color_num', 'skin_type_num', 'hair_color_num',
       'skin_tone_quartile_num.1', 'product_id', 'product_name', 'brand_id',
       'brand_name', 'loves_count', 'rating', 'reviews', 'limited_edition',
       'new', 'online_only', 'out_of_stock', 'sephora_exclusive', 'Award',
       'Clean at Sephora', 'combined_category_Bath & Body',
       'combined_category_Fragrance', 'combined_category_Gifts',
       'combined_category_Hair', 'combined_category_Makeup',
       'combined_category_Men', 'combined_category_Skincare',
       'combined_category_Tools & Brushes',
       'combined_category_Value & Gift Sets', 'Ascorbyl Glucoside', 'Retinol',
       'Ceramide', 'Lactic acid', 'Squalane', 'Aloe', 'Salicylic Acid', 'Shea',
       'Vitamin E/tocopherols', 'jojoba', 'peppermint', 'honey/miel',
       'Phenoxyethanol', 'Rose'],
      dtype='object')

In [5]:


df=result_df.drop(labels=['product_name', 'brand_id', 'brand_name'],axis=1)
df = df.rename(columns={'rating': 'rating_y'})
column_list=df.columns.tolist()
df.size

356748

In [6]:
dense_cols=["rating_y","loves_count","reviews"]
rm_list=["rating_y","loves_count","reviews","product_id"]

for ele in rm_list:
  print(ele)
  column_list.remove(ele)

cat_cols=column_list
print(df.size)
df.dropna()
print(len(cat_cols))

rating_y
loves_count
reviews
product_id
356748
38


In [7]:
df.size

356748

In [8]:
from sklearn import preprocessing
df[dense_cols] = preprocessing.StandardScaler().fit_transform(df[dense_cols])
mappings = {
    col: dict(zip(values, range(len(values))))
    for col, values in map(lambda col: (col, df[col].unique()), cat_cols)
}
for col, mapping in mappings.items():
    df[col] = df[col].map(mapping.get)

In [9]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
ds = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices((
        tf.cast(df[dense_cols].values, tf.float32,),
        tf.cast(df[cat_cols].values, tf.int32,),
    )),
    tf.data.Dataset.from_tensor_slices((
        tf.cast(df[dense_cols].values, tf.float32,),
        tf.cast(df[cat_cols].values, tf.int32,),
    ))
))
ds

<_ZipDataset element_spec=((TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(38,), dtype=tf.int32, name=None)), (TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(38,), dtype=tf.int32, name=None)))>

In [10]:
model=tf.keras.models.load_model('/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/saved_model/my_model')
batch_size = 32  # You can choose an appropriate batch size
ds = ds.batch(batch_size)
res=pd.DataFrame(model.predict(ds))
product_index=res[0].nlargest(10).index
product_index

266/266 [==============================] - 5s 5ms/step


Int64Index([3623, 6242, 415, 422, 2250, 2123, 3687, 838, 3179, 1493], dtype='int64')

In [12]:
res=model.predict([tf.cast(df[dense_cols].values, tf.float32,),
        tf.cast(df[cat_cols].values, tf.int32,)])

266/266 [==============================] - 2s 6ms/step


In [13]:
len(res)

8494

In [16]:
product_index=pd.DataFrame(res)[0].nlargest(10).index
product_index
product_index=pd.DataFrame(res)[0].nlargest(10)

In [17]:
product_index

3623    0.998184
6242    0.995964
415     0.995534
422     0.995149
2250    0.995103
2123    0.995099
3687    0.994972
838     0.994882
3179    0.994614
1493    0.994397
Name: 0, dtype: float32